In [1]:
import sys
import os

thesis_path = "/" + os.path.join(
    *os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [2]:
from evaluations.content_02 import get_per_sample_metrics

metrics_df = get_per_sample_metrics()

In [3]:
metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130766400 entries, 0 to 130766399
Data columns (total 18 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   seed            int64 
 1   checkpoint      object
 2   gazetteer_size  int64 
 3   error_ratio     int64 
 4   error_part      object
 5   timestep        int64 
 6   dataset         object
 7   doc_id          object
 8   targets         int64 
 9   entity_type     object
 10  tp              int64 
 11  fn              int64 
 12  fp              int64 
 13  error_type1     int64 
 14  error_type2     int64 
 15  error_type3     int64 
 16  error_type4     int64 
 17  error_type5     int64 
dtypes: int64(13), object(5)
memory usage: 17.5+ GB


## Performance table 1

Rows: Dimensions:  "gazetteer_size", "error_ratio", "error_part"
Columns: Timestep:

In [4]:
from evaluations.content_02 import aggregate_performance_metrics

agg_df = aggregate_performance_metrics(metrics_df).reset_index()
agg_df = agg_df[agg_df["dataset"]=="lowner_test"]
agg_df

,seed,gazetteer_size,error_ratio,error_part,timestep,dataset,fn,fp,tp,precision,recall,f1
1,1,2000,0,both,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
4,1,2000,0,both,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
7,1,2000,0,both,2,lowner_test,5826,4898,37890,88.552865,86.673072,87.602885
10,1,2000,0,gazetteer,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
13,1,2000,0,gazetteer,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
...,...,...,...,...,...,...,...,...,...,...,...,...
1282,3,8000,15,gazetteer,1,lowner_test,6626,4930,37090,88.267492,84.843078,86.521415
1285,3,8000,15,gazetteer,2,lowner_test,5762,4435,37954,89.537380,86.819471,88.157482
1288,3,8000,15,train,0,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648
1291,3,8000,15,train,1,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648


In [5]:
from evaluations.utils import merge_mean_std, LATEX_METRIC_NAMES
size_agg_df = agg_df.pivot_table(values=["f1", "precision", "recall"],
                                 index=["timestep", "gazetteer_size"],
                                 aggfunc=["mean", "std"]).swaplevel(
                                     0, 1, axis=1).sort_index(axis=1)

metric_names = ["f1", "precision", "recall"]

new_columns = []
for metric_name in metric_names:
    column = (LATEX_METRIC_NAMES[metric_name],
                    "")
    new_columns.append(column)
    size_agg_df[column] = merge_mean_std(size_agg_df,
                                        (metric_name, "mean"),
                                        (metric_name, "std"))
size_agg_df["Dimension"] = "Gazetteer size"
size_agg_df.reset_index(inplace=True)
size_agg_df.set_index(["timestep", "Dimension", "gazetteer_size"], inplace=True)
size_agg_df[new_columns]

Micro-F1   Precision       Recall
                                                                            
timestep Dimension      gazetteer_size                                      
0        Gazetteer size 2000            85.05 (108)  87.60 (23)  82.68 (200)
                        4000            85.26 (155)  87.84 (18)  82.86 (279)
                        6000            85.64 (113)  87.98 (25)  83.43 (200)
                        8000             85.83 (89)  88.14 (21)  83.65 (162)
1        Gazetteer size 2000            85.07 (108)  87.61 (22)  82.69 (199)
                        4000            85.27 (153)  87.86 (19)  82.88 (277)
                        6000            85.66 (113)  88.00 (25)  83.46 (201)
                        8000             85.86 (88)  88.18 (21)  83.68 (160)
2        Gazetteer size 2000            86.77 (109)  88.90 (25)  84.76 (208)
                        4000            87.03 (154)  89.19 (18)  85.03 (282)
                        6000            87.46 (114)  89.35 (25)  85.67 (207)
                        8000             87.65 (91)  89.51 (23)  85.87 (164)

In [6]:
ratio_agg_df = agg_df.pivot_table(values=["f1", "precision", "recall"],
                                  index=["timestep", "error_ratio"],
                                  aggfunc=["mean", "std"]).swaplevel(
                                      0, 1, axis=1).sort_index(axis=1)
timesteps = [0, 1, 2]
metric_names = ["f1", "precision", "recall"]

new_columns = []
for metric_name in metric_names:
    column = (LATEX_METRIC_NAMES[metric_name], "")
    new_columns.append(column)
    ratio_agg_df[column] = merge_mean_std(ratio_agg_df,
                                            (metric_name, "mean"),
                                            (metric_name, "std"))
ratio_agg_df["Dimension"] = "Error ratio"
ratio_agg_df.reset_index(inplace=True)
ratio_agg_df.set_index(["timestep", "Dimension", "error_ratio"], inplace=True)
ratio_agg_df[new_columns]

Micro-F1   Precision       Recall
                                                                      
timestep Dimension   error_ratio                                      
0        Error ratio 0             86.25 (27)  87.94 (33)   84.62 (31)
                     5             85.69 (84)  87.91 (26)  83.60 (148)
                     10            85.38 (67)  87.90 (29)  83.01 (117)
                     15           84.45 (174)  87.82 (29)  81.39 (313)
1        Error ratio 0             86.25 (27)  87.94 (33)   84.62 (31)
                     5             85.70 (84)  87.92 (27)  83.61 (147)
                     10            85.41 (68)  87.92 (29)  83.04 (118)
                     15           84.50 (175)  87.87 (31)  81.44 (313)
2        Error ratio 0             88.05 (32)  89.29 (34)   86.85 (39)
                     5             87.48 (84)  89.24 (31)  85.80 (149)
                     10            87.16 (68)  89.27 (32)  85.16 (120)
                     15           86.22 (176)  89.16 (31)  83.52 (320)

In [7]:
part_agg_df = agg_df.pivot_table(values=["f1", "precision", "recall"],
                                 index=["timestep", "error_part"],
                                 aggfunc=["mean", "std"]).swaplevel(
                                     0, 1, axis=1).sort_index(axis=1)
metric_names = ["f1", "precision", "recall"]

new_columns = []
for metric_name in metric_names:
    column = (LATEX_METRIC_NAMES[metric_name], "")
    new_columns.append(column)
    part_agg_df[column] = merge_mean_std(part_agg_df,
                                            (metric_name, "mean"),
                                            (metric_name, "std"))
part_agg_df["Dimension"] = "Error part"
part_agg_df.reset_index(inplace=True)
part_agg_df.set_index(["timestep", "Dimension", "error_part"], inplace=True)
part_agg_df[new_columns]

Micro-F1   Precision       Recall
                                                                    
timestep Dimension  error_part                                      
0        Error part both        85.08 (126)  87.86 (29)  82.50 (224)
                    gazetteer    86.20 (30)  87.93 (30)   84.53 (38)
                    train       85.05 (139)  87.88 (29)  82.43 (248)
1        Error part both        85.12 (124)  87.89 (30)  82.54 (221)
                    gazetteer    86.23 (31)  87.97 (31)   84.56 (39)
                    train       85.05 (139)  87.88 (29)  82.43 (248)
2        Error part both        86.86 (127)  89.20 (33)  84.67 (228)
                    gazetteer    87.98 (33)  89.29 (33)   86.72 (42)
                    train       86.84 (142)  89.23 (31)  84.62 (258)

In [8]:
import pandas as pd

perf_df = pd.concat([
    size_agg_df[new_columns], ratio_agg_df[new_columns],
    part_agg_df[new_columns]
])
perf_df.sort_index(inplace=True)


In [9]:
perf_df.index.rename(["Timestep", "Dimension", "Value"], inplace=True)
perf_df.columns.rename(["Metric", None], inplace=True)
print(perf_df.style.to_latex(
    siunitx=True,
    hrules=True,
    multirow_align="t",
    column_format=
    "@{}lll*3{S[table-format = 2.2 (2)]}@{} "
))


\begin{tabular}{@{}lll*3{S[table-format = 2.2 (2)]}@{}}
\toprule
{} & {} & {Metric} & {Micro-F1} & {Precision} & {Recall} \\
{} & {} & {} & {} & {} & {} \\
{Timestep} & {Dimension} & {Value} & {} & {} & {} \\
\midrule
\multirow[t]{11}{*}{0} & \multirow[t]{3}{*}{Error part} & both & 85.08 (126) & 87.86 (29) & 82.50 (224) \\
 &  & gazetteer & 86.20 (30) & 87.93 (30) & 84.53 (38) \\
 &  & train & 85.05 (139) & 87.88 (29) & 82.43 (248) \\
 & \multirow[t]{4}{*}{Error ratio} & 0 & 86.25 (27) & 87.94 (33) & 84.62 (31) \\
 &  & 5 & 85.69 (84) & 87.91 (26) & 83.60 (148) \\
 &  & 10 & 85.38 (67) & 87.90 (29) & 83.01 (117) \\
 &  & 15 & 84.45 (174) & 87.82 (29) & 81.39 (313) \\
 & \multirow[t]{4}{*}{Gazetteer size} & 2000 & 85.05 (108) & 87.60 (23) & 82.68 (200) \\
 &  & 4000 & 85.26 (155) & 87.84 (18) & 82.86 (279) \\
 &  & 6000 & 85.64 (113) & 87.98 (25) & 83.43 (200) \\
 &  & 8000 & 85.83 (89) & 88.14 (21) & 83.65 (162) \\
\multirow[t]{11}{*}{1} & \multirow[t]{3}{*}{Error part} & both & 85.12 

## Second Performance table
1. (Data part = Both)
2. Timestep = 3
3. index = Data part, Error rate
4. columns = size
values = f1

In [10]:
from evaluations.content_02 import aggregate_performance_metrics

agg_df = aggregate_performance_metrics(metrics_df).reset_index()
agg_df = agg_df[agg_df["dataset"] == "lowner_test"]
agg_df

,seed,gazetteer_size,error_ratio,error_part,timestep,dataset,fn,fp,tp,precision,recall,f1
1,1,2000,0,both,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
4,1,2000,0,both,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
7,1,2000,0,both,2,lowner_test,5826,4898,37890,88.552865,86.673072,87.602885
10,1,2000,0,gazetteer,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
13,1,2000,0,gazetteer,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
...,...,...,...,...,...,...,...,...,...,...,...,...
1282,3,8000,15,gazetteer,1,lowner_test,6626,4930,37090,88.267492,84.843078,86.521415
1285,3,8000,15,gazetteer,2,lowner_test,5762,4435,37954,89.537380,86.819471,88.157482
1288,3,8000,15,train,0,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648
1291,3,8000,15,train,1,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648


In [20]:
from evaluations.utils import merge_mean_std

perf_df = agg_df[agg_df["error_part"] == "gazetteer"].pivot_table(
    values="f1",
    index=["timestep", "gazetteer_size"],
    columns="error_ratio",
    aggfunc=["mean",
             "std"]).swaplevel(0, 1,
                               axis=1).sort_index(axis=1).sort_index(axis=0)

ratios = [0, 5, 10, 15]
new_columns = []
for ratio in ratios:
    column = (ratio, "")
    new_columns.append(column)
    perf_df[column] = merge_mean_std(perf_df, (ratio, "mean"), (ratio, "std"))

perf_df = perf_df[new_columns]
perf_df.index.rename(["Timestep", "Gazetteer size"], inplace=True)
perf_df.columns.rename(["Error ratio", None], inplace=True)
print(
    perf_df.style.to_latex(
        siunitx=True,
        hrules=True,
        multirow_align="t",
        column_format="@{}ll*4{S[table-format = 2.2 (2)]}@{} "))


\begin{tabular}{@{}ll*4{S[table-format = 2.2 (2)]}@{}}
\toprule
{} & {Error ratio} & {0} & {5} & {10} & {15} \\
{} & {} & {} & {} & {} & {} \\
{Timestep} & {Gazetteer size} & {} & {} & {} & {} \\
\midrule
\multirow[t]{4}{*}{0} & 2000 & 85.86 (5) & 85.77 (21) & 85.92 (8) & 85.76 (23) \\
 & 4000 & 86.24 (10) & 86.11 (11) & 85.91 (19) & 86.24 (3) \\
 & 6000 & 86.40 (9) & 86.22 (9) & 86.42 (12) & 86.30 (11) \\
 & 8000 & 86.49 (26) & 86.45 (34) & 86.52 (4) & 86.55 (12) \\
\multirow[t]{4}{*}{1} & 2000 & 85.86 (5) & 85.78 (18) & 85.97 (1) & 85.80 (23) \\
 & 4000 & 86.25 (10) & 86.11 (12) & 85.94 (23) & 86.29 (1) \\
 & 6000 & 86.40 (9) & 86.23 (14) & 86.48 (12) & 86.40 (9) \\
 & 8000 & 86.49 (26) & 86.51 (33) & 86.55 (4) & 86.63 (13) \\
\multirow[t]{4}{*}{2} & 2000 & 87.60 (7) & 87.48 (31) & 87.58 (11) & 87.47 (21) \\
 & 4000 & 87.99 (14) & 87.93 (4) & 87.77 (2) & 88.08 (11) \\
 & 6000 & 88.25 (7) & 88.06 (17) & 88.24 (10) & 88.12 (11) \\
 & 8000 & 88.38 (20) & 88.33 (17) & 88.25 (16) & 88.23 

# Third performance table
1. Timesteps
3. index = Data part, 
4. columns = Error rate
values = f1

In [21]:
from evaluations.content_02 import aggregate_performance_metrics

agg_df = aggregate_performance_metrics(metrics_df).reset_index()
agg_df = agg_df[agg_df["dataset"] == "lowner_test"]
agg_df

,seed,gazetteer_size,error_ratio,error_part,timestep,dataset,fn,fp,tp,precision,recall,f1
1,1,2000,0,both,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
4,1,2000,0,both,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
7,1,2000,0,both,2,lowner_test,5826,4898,37890,88.552865,86.673072,87.602885
10,1,2000,0,gazetteer,0,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
13,1,2000,0,gazetteer,1,lowner_test,6805,5412,36911,87.212627,84.433617,85.800625
...,...,...,...,...,...,...,...,...,...,...,...,...
1282,3,8000,15,gazetteer,1,lowner_test,6626,4930,37090,88.267492,84.843078,86.521415
1285,3,8000,15,gazetteer,2,lowner_test,5762,4435,37954,89.537380,86.819471,88.157482
1288,3,8000,15,train,0,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648
1291,3,8000,15,train,1,lowner_test,8050,4772,35666,88.199219,81.585689,84.763648


In [27]:
perf_df = agg_df[agg_df["gazetteer_size"] == 8000].pivot_table(
    values="f1",
    index=["timestep", "error_part"],
    columns="error_ratio",
    aggfunc=["mean",
             "std"]).swaplevel(0, 1,
                               axis=1).sort_index(axis=1).sort_index(axis=0)
ratios = [0, 5, 10, 15]
new_columns = []
for ratio in ratios:
    column = (ratio, "")
    new_columns.append(column)
    perf_df[column] = merge_mean_std(perf_df, (ratio, "mean"), (ratio, "std"))

perf_df = perf_df[new_columns]
perf_df.index.rename(["Timestep", "Error part"], inplace=True)
perf_df.columns.rename(["Error ratio", None], inplace=True)
print(
    perf_df.style.to_latex(
        siunitx=True,
        hrules=True,
        multirow_align="t",
        column_format="@{}ll*4{S[table-format = 2.2 (2)]}@{} "))

\begin{tabular}{@{}ll*4{S[table-format = 2.2 (2)]}@{}}
\toprule
{} & {Error ratio} & {0} & {5} & {10} & {15} \\
{} & {} & {} & {} & {} & {} \\
{Timestep} & {Error part} & {} & {} & {} & {} \\
\midrule
\multirow[t]{3}{*}{0} & both & 86.49 (26) & 85.86 (40) & 85.37 (5) & 84.19 (96) \\
 & gazetteer & 86.49 (26) & 86.45 (34) & 86.52 (4) & 86.55 (12) \\
 & train & 86.49 (26) & 85.88 (1) & 85.35 (3) & 84.30 (63) \\
\multirow[t]{3}{*}{1} & both & 86.49 (26) & 85.90 (41) & 85.44 (6) & 84.33 (98) \\
 & gazetteer & 86.49 (26) & 86.51 (33) & 86.55 (4) & 86.63 (13) \\
 & train & 86.49 (26) & 85.88 (1) & 85.35 (3) & 84.30 (63) \\
\multirow[t]{3}{*}{2} & both & 88.38 (20) & 87.66 (34) & 87.22 (15) & 85.86 (94) \\
 & gazetteer & 88.38 (20) & 88.33 (17) & 88.25 (16) & 88.23 (6) \\
 & train & 88.38 (20) & 87.75 (8) & 87.15 (15) & 86.20 (67) \\
\bottomrule
\end{tabular}



# Collect data metrics

## Collect gazetteer data

In [1]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [3]:
from glob import glob

list(
    glob(os.path.join(thesis_path, "experiments", "02_content", "data", r"**",
                      "01_search_results", "**", r"*.pkl"),
         recursive=True))


['/home/loebbert/projects/thesis/experiments/02_content/data/01_search_results/search_results_dev.pkl',
 '/home/loebbert/projects/thesis/experiments/02_content/data/01_search_results/search_results_train.pkl',
 '/home/loebbert/projects/thesis/experiments/02_content/data/01_search_results/search_results_test.pkl',
 '/home/loebbert/projects/thesis/experiments/02_content/data/seed_2/01_search_results/size_6000/error_ratio_5/error_search_results_dev.pkl',
 '/home/loebbert/projects/thesis/experiments/02_content/data/seed_2/01_search_results/size_6000/error_ratio_5/error_search_results_test.pkl',
 '/home/loebbert/projects/thesis/experiments/02_content/data/seed_2/01_search_results/size_6000/error_ratio_5/sampled_search_results_error_train.pkl',
 '/home/loebbert/projects/thesis/experiments/02_content/data/seed_2/01_search_results/size_6000/error_ratio_5/sampled_search_results_error_dev.pkl',
 '/home/loebbert/projects/thesis/experiments/02_content/data/seed_2/01_search_results/size_6000/error_

In [6]:
from typing import Dict
from itertools import product

search_results: Dict[str, str] = {}
seeds = [1, 2, 3]
sizes = [2000, 4000, 6000, 8000]
error_ratios = [0, 5, 10, 15]
error_parts = ["train", "gazetteer", "both"]
parts = ["train", "dev", "test"]
timesteps = [0, 1, 2]
datapath = os.path.join(thesis_path, "experiments", "02_content", "data")
for seed, size, error_ratio, error_part, timestep, part  in product(
        seeds,
        sizes,
        error_ratios,
        error_parts,
        timesteps, parts
):
    search_base_path = os.path.join(datapath, f"seed_{str(seed)}",
                                    "01_search_results",
                                    f"size_{size}",
                                    f"error_ratio_{error_ratio}")
    if timestep == 0:
        if error_part in ["both", "gazetteer"]:
            if error_part == "both" and part in ["train", "dev"]:
                search_results[
                    f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                        search_base_path, f"error_search_results_error_{part}.pkl")
            else:
                search_results[
                    f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                        search_base_path, f"error_search_results_{part}.pkl")
        else:
            if part in ["train", "dev"]:
                search_results[
                    f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                        search_base_path, f"sampled_search_results_error_{part}.pkl")
            else:
                search_results[
                    f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                        search_base_path, f"sampled_search_results_{part}.pkl")

    if timestep == 1:
        search_results[
            f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                search_base_path, f"sampled_search_results_{part}.pkl")

    if timestep == 2:
        search_results[
            f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                datapath, "01_search_results", f"search_results_{part}.pkl")
        
len(search_results)


1296

In [8]:
from typing import Dict
from itertools import product

labeled_data: Dict[str, str] = {}
seeds = [1, 2, 3]
sizes = [2000, 4000, 6000, 8000]
error_ratios = [0, 5, 10, 15]
error_parts = ["train", "gazetteer", "both"]
parts = ["train", "dev", "test"]
timesteps = [0, 1, 2]
datapath = os.path.join(thesis_path, "experiments", "02_content", "data")
for seed, size, error_ratio, error_part, timestep, part in product(
        seeds, sizes, error_ratios, error_parts, timesteps, parts):
    labeled_data_path = os.path.join(datapath, f"seed_{str(seed)}",
                                    "00_datasets", f"size_{size}",
                                    f"error_ratio_{error_ratio}")
    if timestep == 0:
        if error_part == "train" and part in ["train", "dev"]:
            labeled_data[
                f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                    labeled_data_path,
                    f"error_lowner_{part}.json")
        else:
            labeled_data[
                f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                    thesis_path, "data", "mlowner", f"lowner_{part}.json")

    if timestep > 0:
        labeled_data[
            f"{seed}_{size}_{error_ratio}_{error_part}_{timestep}_{part}"] = os.path.join(
                thesis_path, "data", "mlowner", f"lowner_{part}.json")
len(labeled_data)

1296

In [1]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [2]:
from evaluations.content_02 import get_search_results_entity_coverage_per_sample

eecr_search_results = get_search_results_entity_coverage_per_sample()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Compute EECR search results:   0%|          | 1/1296 [05:23<116:20:26, 323.42s/it]/home/loebbert/miniconda3/envs/haystack/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fbbd8877520>
  self.pid = os.fork()
/home/loebbert/miniconda3/envs/haystack/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fbbd8877520>
  self.pid = os.fork()
/home/loebbert/miniconda3/envs/haystack/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fbbd8877520>
  self.pid = os.fork()
/home/loebbert/miniconda3/envs/haystack/lib/python3.10/multiprocessing/popen_fork.py:6

In [3]:
eecr_search_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21794400 entries, 0 to 21794399
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   seed            object 
 1   gazetteer_size  object 
 2   error_ratio     object 
 3   error_part      object 
 4   timestep        object 
 5   dataset         object 
 6   doc_id          object 
 7   targets         int64  
 8   eecr            float64
 9   ρ=1             int64  
 10  ρ ∈ (0.5,1)     int64  
 11  ρ ∈ (0,0.5]     int64  
 12  ρ=0∧C≠0         int64  
 13  ρ=0∧C=0         int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 2.3+ GB


In [ ]:
from evaluations.content_02 import get_search_results_data_ccr_metrics

ccr_search_results = get_search_results_data_ccr_metrics()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CCR search results:   0%|          | 0/1296 [00:00<?, ?it/s]/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
CCR search results:   0%|          | 2/1296 [00:53<8:39:06, 24.07s/it] 

[0, 9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144, 153, 162, 171]
[(1, 9), (10, 18), (19, 27), (28, 36), (37, 45), (46, 54), (55, 63), (64, 72), (73, 81), (82, 90), (91, 99), (100, 108), (109, 117), (118, 126), (127, 135), (136, 144), (145, 153), (154, 162), (163, 171), (172, 180)]


## Collect datasets

In [1]:
import sys
import os

thesis_path = "/" + os.path.join(*os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)

In [2]:
from evaluations.content_02 import get_labeled_data_entity_coverage

eecr_labeled_data_df = get_labeled_data_entity_coverage()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Compute EECR labeled data:   0%|          | 1/1296 [10:59<237:22:37, 659.89s/it]
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f347323e5c0>


In [3]:
eecr_labeled_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   seed            1296 non-null   object 
 1   gazetteer_size  1296 non-null   object 
 2   error_ratio     1296 non-null   object 
 3   error_part      1296 non-null   object 
 4   timestep        1296 non-null   object 
 5   dataset         1296 non-null   object 
 6   eecr            1296 non-null   float64
 7   ρ=1             1296 non-null   int64  
 8   ρ ∈ (0.5,1)     1296 non-null   int64  
 9   ρ ∈ (0,0.5]     1296 non-null   int64  
 10  ρ=0∧C≠0         1296 non-null   int64  
 11  ρ=0∧C=0         1296 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 121.6+ KB


In [2]:
from evaluations.content_02 import get_labeled_data_entity_coverage_per_sample

eecr_per_sample_labeled_data = get_labeled_data_entity_coverage_per_sample()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Compute EECR labeled data:   0%|          | 1/1296 [11:34<249:42:35, 694.17s/it]
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f7dd23a99c0>


In [3]:
eecr_per_sample_labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21794400 entries, 0 to 21794399
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   seed            object 
 1   gazetteer_size  object 
 2   error_ratio     object 
 3   error_part      object 
 4   timestep        object 
 5   dataset         object 
 6   doc_id          object 
 7   targets         int64  
 8   eecr            float64
 9   ρ=1             int64  
 10  ρ ∈ (0.5,1)     int64  
 11  ρ ∈ (0,0.5]     int64  
 12  ρ=0∧C≠0         int64  
 13  ρ=0∧C=0         int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 2.3+ GB


In [4]:
from evaluations.content_02 import get_search_results_data

search_results, datasets = get_search_results_data()

In [9]:
from collections import defaultdict
search_result_comb = defaultdict(list)
for key, search_result in search_results.items():
    search_result_comb[(search_result, datasets[key])].append(key)
list(search_result_comb)

[('/home/loebbert/projects/thesis/experiments/02_content/data/seed_1/01_search_results/size_2000/error_ratio_0/sampled_search_results_error_train.pkl',
  '/home/loebbert/projects/thesis/experiments/02_content/data/seed_1/00_datasets/size_2000/error_ratio_0/error_lowner_train.json'),
 ('/home/loebbert/projects/thesis/experiments/02_content/data/seed_1/01_search_results/size_2000/error_ratio_0/sampled_search_results_error_dev.pkl',
  '/home/loebbert/projects/thesis/experiments/02_content/data/seed_1/00_datasets/size_2000/error_ratio_0/error_lowner_dev.json'),
 ('/home/loebbert/projects/thesis/experiments/02_content/data/seed_1/01_search_results/size_2000/error_ratio_0/sampled_search_results_test.pkl',
  '/home/loebbert/projects/thesis/data/mlowner/lowner_test.json'),
 ('/home/loebbert/projects/thesis/experiments/02_content/data/seed_1/01_search_results/size_2000/error_ratio_0/sampled_search_results_train.pkl',
  '/home/loebbert/projects/thesis/data/mlowner/lowner_train.json'),
 ('/home/l